<a href="https://colab.research.google.com/github/parampopat/Wiki-Movies/blob/master/movie_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
__author__ = "Param Popat"
__version__ = "1"
__git__ = "https://github.com/parampopat/"
"""

'\n__author__ = "Param Popat"\n__version__ = "1"\n__git__ = "https://github.com/parampopat/"\n'

In [2]:
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import pandas as pd

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# load ascii text and covert to lowercase
movieNames = pd.read_csv("/content/drive/My Drive/Wiki-Movies/Plot_associations.csv").iloc[:, 1].values
raw_text = ""
for filename in movieNames:
    raw_text += open("/content/drive/My Drive/Wiki-Movies/Plot/" + filename + '.txt').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [5]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print(
    "Total Characters: ", n_chars)
print(
    "Total Vocab: ", n_vocab)

Total Characters:  84100
Total Vocab:  54


In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Patterns:  84000


In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath = "/content/drive/My Drive/Wiki-Movies/trained/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
84000/84000 [==============================] - 349s 4ms/step - loss: 2.9252

Epoch 00001: loss improved from inf to 2.92522, saving model to /content/drive/My Drive/Wiki-Movies/trained/weights-improvement-01-2.9252-bigger.hdf5
Epoch 2/50
84000/84000 [==============================] - 343s 4ms/step - loss: 2.7228

Epoch 00002: loss improved from 2.92522 to 2.72276, saving model to /content/drive/My Drive/Wiki-Movies/trained/weights-improvement-02-2.7228-bigger.hdf5
Epoch 3/50
84000/84000 [==============================] - 343s 4ms/step - loss: 2.6027

Epoch 00003: loss improved from 2.72276 to 2.60267, saving model to /content/drive/My Drive/Wiki-Movies/trained/weights-improvement-03-2.6027-bigger.hdf5
Epoch 4/50
84000/84000 [===================

In [0]:
start = numpy.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")